---
## Ch.2　ニューラルネットの基礎

---
### 2.1　教師あり学習
入力変数ベクトル $\mathbf{x} \in \mathcal{X}$、出力変数 $y \in \mathcal{Y}$ について  
ある変数 $\mathbf{x}$ が与えられた時に $y$ を予測するモデルを学習することを目的とする。

教師あり学習の定義
$|\mathcal{D}|$ 個の訓練事例を $\mathcal{D} = \{(\mathbf{x}^{(n)}, y^{(n)})\}_{n=1}^{|\mathcal{D}|}$ としたとき、  
訓練データでの誤差（以下の目的関数）を最小化するモデルパラメータ $\mathbf{\theta}$ の値を求める手続きを教師あり学習と呼ぶ。

\begin{align*}
L(\mathbf{\theta}) = \frac{1}{|\mathcal{D}|} \sum_{n=1}^{|\mathcal{D}|} l_{\mathbf{\theta}} (\mathbf{x}^{(n)}, y^{(n)})
\end{align*}

自然言語処理では出力は単語のように離散集合 $\mathcal{Y} = \{1, \ldots, |\mathcal{Y}| \}$ であることが多く、分類問題として定式化できる。  
分類問題では 0-1損失関数の代理損失（surrogate loss）として、  
次の交差エントロピー損失関数と（多クラス）ヒンジ損失関数がよく用いられる。

・cross-entropy loss

\begin{align*}
l_{\mathbf{\theta}}^{\text{cross-entropy}} \left( \mathbf{x}^{(n)}, y^{(n)} \right) = - \log \frac{\exp ( f_{\mathbf{\theta}}( \mathbf{x}^{(n)}, y^{(n)} ))}{\sum_{\tilde{y} \in \mathcal{Y}} \exp ( f_{\mathbf{\theta}}(\mathbf{x}^{(n)}, \tilde{y}))}
\end{align*}

・multiclass hinge loss

\begin{align*}
l_{\theta}^{\text{hinge}}\left( \mathbf{x}^{(n)}, y^{(n)} \right) = \max \left( 0, \, 1 - f_{\theta}\left( \mathbf{x}^{(n)}, y^{(n)} \right) + \max_{\tilde{y} \in {\mathcal{Y} \\ \{y^{(n)}\}} } f_{\theta}( \mathbf{x}^{(n)}, \tilde{y}) \right)
\end{align*}

---
$f_{\theta}(\mathbf{x},y)$ は $\theta$ をパラメータとするスコア関数で、$\hat{y} = \operatorname{argmax}_{y} f_{\theta}(\mathbf{x}^{(n)}, y)$ を予測として使用することを想定している。  
注記なき限りスコア関数はニューラルネットによる実装とする。

誤差関数として交差エントロピー、スコア関数 $f_{\theta}(\mathbf{x},y)$ を線形モデルとするとき多項ロジスティック回帰という。  
交差エントロピー損失関数は以下の条件付き確率モデルを想定し、  
その負の対数尤度を想定していると考えることもできる。  

\begin{align*}
P_{\theta}(y|\mathbf{x}) = \frac{\exp ( f_{\theta} (\mathbf{x},y))}{\sum_{\tilde{y} \in \mathcal{Y}} \exp ( f_{\theta} ( \mathbf{x}, \tilde{y}))}
\end{align*}

交差エントロピー損失関数は、真の分布 $P^{*}_{\theta}(y|\mathbf{x})$ とモデル $P_{\theta}(y|\mathbf{x})$ との距離を表す交差エントロピーを  
訓練データで経験近似していることに相当することからそう呼ばれる。

ソフトマックス関数

\begin{align*}
\operatorname{softmax}(\mathbf{o})_{y} = \frac{\exp (o_{y})}{\sum_{\tilde{y} \in \mathcal{Y}} \exp (o_{y})}
\end{align*}

を用いると、上の式は

\begin{align*}
P_{\theta}(y|\mathbf{x}) = \operatorname{softmax}(\mathbf{o})_{y}
\end{align*}

と書くことができる。

---
### 2.2　順伝播型ニューラルネット
非線形変換を $L$ 回繰り返すFFNN (feedforward neural network) を想定する。  
$l$ 回目の変換結果の出力ベクトル $\mathbf{h}^{(l)} \in \mathcal{R}^{N^{(1)}}$ を隠れ状態ベクトルと呼ぶ。  

出力層からの出力候補 $y$ に対するスコアの計算は $|\mathcal{Y}| \times N^{(L)}$ のパラメータ行列 $\mathbf{W}^{(o)}$ と  
バイアスパラメータベクトル $\mathbf{b}^{(o)}$ を使って以下のように表す。

\begin{align*}
\mathbf{o} = \mathbf{W}^{(o)}\mathbf{h}^{(L)} + \mathbf{b}^{(o)}
\end{align*}

次に、各層の隠れ状態ベクトル $\mathbf{h}^{(l)} \, (l=1,\ldots,L)$ は活性化関数を $a$ として

\begin{align*}
\mathbf{h}^{(l)} = a^{(l)} \left( \mathbf{W}^{(l)}\mathbf{h}^{(l-1)} + \mathbf{b}^{(l)} \right)
\end{align*}

と表される。ここで $l=0$ のとき $\mathbf{h}^{0} = \mathbf{x}$ （$x$ は入力）とする。

---
### 2.3　活性化関数
シグモイド関数、tanh、ReLUなどがよく用いられる。  
活性化関数は非線形でなければならない。

---
### 2.4　勾配法
学習率を $\eta$ とするとき、次の更新式によって学習する。

\begin{align*}
\theta^{(k+1)} = \theta^{(k)} - \eta \partial L(\theta^{(k)})
\end{align*}

勾配法は局所的な最急降下方向に $\eta$ に比例してパラメータを変化させる。

バッチ法による勾配降下法を行うとき、  
損失関数の偏微分は個々のデータにおける偏微分の和に分解できる。

\begin{align*}
\partial L(\theta)^{batch} \frac{1}{|\mathcal{D}|} \sum_{n=1}^{|\mathcal{D}|} \partial l_{\theta}(\mathbf{x}^{(n)}, y^{(n)})
\end{align*}

バッチ法では訓練データ全てについての計算が必要なことから、  
ランダムに選んだ事例で近似する確率的勾配法がよく用いられる。  
１事例のみを用いるオンライン確率的勾配法では次のように近似する。$i$ はランダムに選んだ事例の番号。  
　（メモ：原文中の式に $i$ 見当たらず）

\begin{align*}
\partial L(\theta)^{\text{online}} = \partial l_{\theta} (\mathbf{x}^{(n)}, y^{(n)})
\end{align*}

ミニバッチ確率的勾配法では次の通り。

\begin{align*}
\partial L(\theta)^{\text{mini-batch}} = \frac{1}{|\mathcal{B}|} \sum_{m=1}^{|\mathcal{B}|} \partial l_{\theta} (\mathbf{x}^{(\mathcal{B}[m])}, y^{(\mathcal{B}[m])})
\end{align*}



---
### 2.5　誤差逆伝播法
